# Dialogue Management

## Two major components 

1. Understand what the user is saying

2. Decide what to say in response

## Language Understanding 

So far, we have been working with regular expressions.

Regular expressions are one technique we can use to program language understanding into a dialogue agent. 

We define what the agent understands by creating regular expressions.

The agent will understand what a user says if it matches the regular expression that is programmed.

If we take the phone type example from earlier:

In [2]:
import re
from typing import List
from re import Pattern

def regex_matcher(regex: Pattern, instr: str) -> List[str]:
    ts = [None] * regex.groups
    
    for t in regex.findall(instr):
        if isinstance(t, str): t = [t]
        for i, literal in enumerate(t):
            if ts[i] is None and literal:
                ts[i] = literal
    
    return ts

re_phone = re.compile(r'(?:\s|^)(apple|google|samsung)|(iphone|pixel|galaxy|android)(?:\s|,|\.|$)')

print(regex_matcher(re_phone, 'yes I have an iphone'))
print(regex_matcher(re_phone, 'yes I have google pixel'))
print(regex_matcher(re_phone, 'yes I have a galaxy phone'))

[None, 'iphone']
['google', 'pixel']
[None, 'galaxy']


We have programmed an agent that can understand when the user is sharing the type of phone that they have by recognizing the different companies and phone models.

Using the regular expression grouping capability, we can also specify what pieces of information we want the dialogue agent to extract if the user shares them.

For the phone type example, we are programming the dialogue agent to pick out the company name and the phone model name in such a way that this information can be stored and accessed later. 

It is important for dialogue agents to have this memory ability in order to make coherent responses to what the user says.

## Response Generation

Each utterance that a user says affects the dialogue situation that the agent is currently in. 

Depending on what the user says, we want the agent to be able to make different responses that make sense for the particular state of the dialogue.

If we take the starting turn of the phone conversation as an example:

In [4]:
from typing import Any
from types import SimpleNamespace

def turn_0(res: SimpleNamespace):
    s = 'S: are you using a smartphone?'
    u = input(s + '\nU: ')

    yn = regex_matcher(res.re_yn, u)
    phone = regex_matcher(res.re_phone, u)
    res.in_phone_company = phone[0]
    res.in_phone_name = phone[1]

    if any(phone):
        turn_1a(res)
    elif yn[0]:
        turn_1b(res)
    elif yn[1]:
        turn_1c(res)

    print('S: good bye!')

There are three different dialogue states that we can be in, depending on what the user says in response to our question.

1. The user has shared the particular type of phone that they have

2. The user has said yes to using a smartphone, but not specified a type

3. The user has said no to using a smartphone

In each of these cases, the agent should take a different course of action when responding to the user.

Since the amount of information that the agent knows about the user is different, its interactions should be tailored to these different situations in order to hold a fluent and natural conversation.

So far, we have been using conditional statements to make these dialogue decisions.

## Dialogue State Machine

There is name for this approach to dialogue: State Machine. 

State Machines are used to model many different processes, including dialogue. 

Anything that can be thought of as consisting of a sequence of actions resulting in different situations can be modelled as a State Machine.

There are two major components of a State Machine:

### 1. States

* Describe a specific point/situation in the process being defined 

### 2. Transitions

* Actions to move from one state to another

For dialogue, 

* transitions are the language being exchanged back and forth from either the agent or the user

* states are the different situations that result from specific sequences of language.

Below is a State Machine of the phone conversation we have been working with. 

* S marks states where the system is giving a response.

* U marks states where the user is giving a response. 

* The arrows show the response (or an approximation of it) that takes you from one state to another.

We will be using a python package that provides a straightforward interface for making state machines for dialogue.

Documentation and installation instructions can be found here: https://pypi.org/project/emora-stdm/

In this package, a state machine for dialogue is called a DialogueFlow.

Here is a simple example:

In [ ]:
from emora_stdm import DialogueFlow
from enum import Enum

# states are typically represented as an enum
class State(Enum):
    START = 0
    FAM_ANS = 1
    FAM_Y = 2
    FAM_N = 3
    FAM_ERR = 4
    WHATEV = 5

# initialize the DialogueFlow object, which uses a state-machine to manage dialogue
df = DialogueFlow(State.START)

# add transitions to create an arbitrary graph for the state machine
df.add_system_transition(State.START, State.FAM_ANS, '[!do you have a $F={brother, sister, son, daughter, cousin}]')
df.add_user_transition(State.FAM_ANS, State.FAM_Y, '[{yes, yea, yup, yep, i do, yeah}]')
df.add_user_transition(State.FAM_ANS, State.FAM_N, '[{no, nope}]')
df.add_system_transition(State.FAM_Y, State.WHATEV, 'thats great i wish i had a $F')
df.add_system_transition(State.FAM_N, State.WHATEV, 'ok then')
df.add_system_transition(State.FAM_ERR, State.WHATEV, 'im not sure i understand')

# each state that will be reached on the user turn should define an error transition if no other transition matches
df.set_error_successor(State.FAM_ANS, State.FAM_ERR)
df.set_error_successor(State.WHATEV, State.START)

if __name__ == '__main__':
    # automatic verification of the DialogueFlow's structure (dumps warnings to stdout)
    df.check()
    # run the DialogueFlow in interactive mode to test
    df.run(debugging=True)

Since we have been focusing on regular expressions up until this point, the first thing we will discuss is how to specify the language expression for each transition between states in the package `emora-stdm`.

It uses a modified version of regular expressions - called `NatEx` - which simplifies much of the syntax, but ultimately the expression that you write is translated into a regular expression.

## NatEx

### Literal
```
'hello there'
```
directly match a literal substring

### Disjunction
```
'{hello there, hi}'
```
matches a substring containing exactly one term inside `{}`, in this case 
"hello there" and "hi" both match.

### Conjunction
```
'<bob, hi>'
```
matches a substring that contains at least all terms inside `<>`,
in this case, "hi bob" and "oh bob well hi there" both would match, but not
"hi"

### Flexible sequence
```
'[hi, bob, how, you]'
```
matches as long as the substring contains all terms inside `[]`,
and the terms are ordered properly within the utterance. Matches
in the example include "hi bob how are you", but not "how are you 
bob". Note that this expression matches any amount of characters
before and after the requisite sequence.

### Inflexible sequence
```
'[!how, are, you]'
```
matches an exact sequence of terms with no words inserted between
terms. The only utterance matching the example is "how are you".
This construct is helpful with nested constructs inside of it that
require an exact ordering, with no extra characters between each
element.

### Negation
```
[!i am -bad]
```
prepend `-` to negate the next term in the expression. The example
will match any expression starting with "i am" where "bad" does NOT
follow. Note that the scope of the negation extends to the end
of the substring due to limitations in regex.

### Regular expression
```
'/[A-Z a-z]+/'
```
substrings within `//` define a python regex directly.

### Nesting
```
'[!{hi, hello} [how, weekend]]'
```
would match "hi how was your weekend", "oh hello so how is the
weekend going", ...

### Variable assignment
```
'[!i am $f={good, bad}]'
```
using `$var=` will assign variable `var` to the next term in
the expression. The variable will persist until overwritten,
and can be referenced in future NLU or NLG expressions.
The example would match either "i am good" or "i am bad", and
assigns variable "f" to either "good" or "bad" depending
on what the user said.

### Variable reference
```
[!why are you $f today]
```
using `$` references a previously assigned variable. If no such
variable exists, the expression as a whole returns with no match.
The example would match "why are you good today" if `f="good"`, 
but would not match if `f="bad"`

Let's return to the phone conversation we have been working with.

In [ ]:
from emora_stdm import NatexNLU

# (?:\s|^)(yes|yeah)(?:\s|,|\.|$)

yes = r"[{yes,yeah}]"
yes_natex = NatexNLU(yes)
print(yes_natex.match("yes i have an iphone"))
print()

In [ ]:
# (?:\s|^)(no|not really)(?:\s|,|\.|$)(?:.*)

no = r"[{no,not really}]"
no_natex = NatexNLU(no)
print(no_natex.match("actually not really"))
print()

In [ ]:
# (?:\s|^)(apple|google|samsung)|(iphone|pixel|galaxy|android)(?:\s|,|\.|$)

pcm_vars = {}
phone_company_model = r"[$company={apple,google,samsung},$model={iphone,pixel,galaxy,android}]" #need to be optional elements (either or both can happen)
phone_company_model_natex = NatexNLU(phone_company_model)
print(phone_company_model_natex.match("the samsung android one", vars=pcm_vars))
print(pcm_vars)
print()

In [ ]:
# (?:\s|^)(\d+)(?:\s|-)+(month|year)

t_vars = {}
timeframe = r"[$number=/\d+/,$time_type={month,year}]" #optional - in between
timeframe_natex = NatexNLU(timeframe)
print(timeframe_natex.match("6 months", vars=t_vars))
print(t_vars)
print()

In [ ]:
# (?:\s|^)(?:since|from)\s(?:(january|february|march|april|may|june|july|august|september|october|november|december)\s)?(\d{2,4})

d_vars = {}
date = r"[!{since,from},$month={january,february,march,april,may,june,july,august,september,october,november,december},$year=/\d{2,4}/]"
date_natex = NatexNLU(date)
print(date_natex.match("since february 2019", vars=d_vars))
print(d_vars)
print()

In [ ]:
# (?:\s|^)(?:iphone|version)\s(\d+s?(?: (?:plus|max))?)(?:\s|,|\.|$)

im_vars = {}
iphone_model = r"[$iphone_model=[!{iphone,version},{/\d+/,/\d+s/},{plus,max}]]"
iphone_model_natex = NatexNLU(iphone_model)
print(iphone_model_natex.match("iphone 10s plus", vars=im_vars))
print(im_vars)
print()